In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
sys.path.insert(0, src_path)

from train import validate_model, train_model, plot_training_history
from datasets import CTDataset
from models import SliceFormer3D, CTClassifier

In [ ]:
batch_size = 1
pre_unfreeze_epochs = 2
post_unfreeze_epochs = 7
num_workers = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random_state = 42

In [ ]:
base_dir = "/kaggle/input/mosmed-cancer-i/MosMedData-LDCT-LUNGCR-type I-v 1"
registry_path = os.path.join(base_dir, "dataset_registry.xlsx")
studies_dir = os.path.join(base_dir, "studies")

df = pd.read_excel(registry_path)

healthy_studies = df[df['pathology'] == 0]['study_instance_anon'].tolist()
cancer_studies = df[df['pathology'] == 1]['study_instance_anon'].tolist()

healthy_patient_dirs = [os.path.join(studies_dir, uid) for uid in healthy_studies]
cancer_patient_dirs = [os.path.join(studies_dir, uid) for uid in cancer_studies]

print(f"Здоровые пациенты: {len(healthy_patient_dirs)}")
print(f"Пациенты с патологией: {len(cancer_patient_dirs)}")

In [ ]:
dir_path = "/kaggle/input/mosmeddata-chest-ct-scans-with-covid19/MosMedData Chest CT Scans with COVID-19 Related Findings COVID19_1110 1.0/studies"
random_state = 42

ct_folders = [f for f in os.listdir(dir_path) if f.startswith("CT-")]

all_samples = []
all_labels = []

for folder in ct_folders:
    folder_path = os.path.join(dir_path, folder)
    nii_paths = glob.glob(os.path.join(folder_path, "*.nii*"))

    label = 0 if folder == "CT-0" else 1

    all_samples.extend(nii_paths)
    all_labels.extend([label] * len(nii_paths))

print(f"Всего образцов: {len(all_samples)}")
print(f"Норма (CT-0): {all_labels.count(0)}")
print(f"Патология: {all_labels.count(1)}")

In [ ]:
paths = healthy_patient_dirs + cancer_patient_dirs + all_samples
labels = [0.0] * len(healthy_patient_dirs) + [1.0] * len(cancer_patient_dirs) + all_labels
ct_format = ['dcm'] * (len(healthy_patient_dirs) + len(cancer_patient_dirs)) + ['nii'] * len(all_samples)

In [ ]:
indices = list(range(len(paths)))

train_val_idx, test_idx = train_test_split(
    indices,
    test_size=0.2,
    stratify=labels,
    random_state=random_state
)


train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.25,
    stratify=[labels[i] for i in train_val_idx],
    random_state=random_state
)

def subset_by_indices(lst, idxs):
    return [lst[i] for i in idxs]

train_paths = subset_by_indices(paths, train_idx)
train_labels = subset_by_indices(labels, train_idx)
train_formats = subset_by_indices(ct_format, train_idx)

val_paths = subset_by_indices(paths, val_idx)
val_labels = subset_by_indices(labels, val_idx)
val_formats = subset_by_indices(ct_format, val_idx)

test_paths = subset_by_indices(paths, test_idx)
test_labels = subset_by_indices(labels, test_idx)
test_formats = subset_by_indices(ct_format, test_idx)

In [ ]:
train_dataset = CTDataset(train_paths, train_labels, train_formats)
val_dataset = CTDataset(val_paths, val_labels, val_formats)
test_dataset = CTDataset(test_paths, test_labels, test_formats)

print(f"\nTrain: {len(train_dataset)} ({len(train_dataset)/len(paths):.1%})")
print(f"Val:   {len(val_dataset)} ({len(val_dataset)/len(paths):.1%})")
print(f"Test:  {len(test_dataset)} ({len(test_dataset)/len(paths):.1%})")

In [ ]:
train_labels_array = np.array(train_labels)
num_neg = (train_labels_array == 0).sum()
num_pos = (train_labels_array == 1).sum()

print(f"Train balance — норма: {num_neg}, патология: {num_pos} (ratio pos/neg = {num_pos/num_neg:.2f})")

class_weights = 1.0 / np.array([num_neg, num_pos])
sample_weights = class_weights[train_labels_array.astype(int)]

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=True
)

In [ ]:
ct_clf = CTClassifier(embed_dim=128)
ct_clf.load_state_dict(
    torch.load(
        "/kaggle/input/sclice-classifier/pytorch/default/1/best_ct_classifier.pth", 
        map_location="cuda"
    )
)

ct_clf.eval()

slice_encoder = ct_clf.encoder

scan_model = SliceFormer3D(
    slice_encoder=slice_encoder,
    embed_dim=128,
    num_heads=4,
    num_transformer_layers=2,
    max_slices=256
)

scan_model.to(device)

In [ ]:
for param in scan_model.slice_encoder.parameters():
    param.requires_grad = False

pre_unfreeze_history = train_model(
    scan_model, 
    train_loader, 
    val_loader, 
    device, 
    epochs=pre_unfreeze_epochs, 
    lr=1e-4,
    wd=1e-5
)

plot_training_history(pre_unfreeze_history)

In [ ]:
for param in scan_model.slice_encoder.parameters():
    param.requires_grad = True

post_unfreeze_history = train_model(
    scan_model, 
    train_loader, 
    val_loader, 
    device, 
    epochs=post_unfreeze_epochs, 
    lr=1e-5,
    wd=1e-6
)

plot_training_history(post_unfreeze_history)


In [ ]:
validate_model(model, test_loader, device)